In [1]:
import pandas as pd
from collections import Counter

from nltk.tokenize import word_tokenize
import sys
input = "Hotel_Reviews.csv"

In [2]:
df = pd.read_csv(input, sep = ',')


In [4]:
neg_revs = []
pos_revs = []
for index, row in df.iterrows():
    if row['Negative_Review'] != 'No Negative':
        neg_revs.append(row['Negative_Review'])
    if row['Positive_Review'] != 'No Positive':
        pos_revs.append(row['Positive_Review'])
print(len(neg_revs))
print(len(pos_revs))
train_pos = pos_revs[0:int(0.75*len(neg_revs))]
test_pos = pos_revs[int(0.75*len(neg_revs)):len(neg_revs)]
train_neg = neg_revs[0:int(0.75*len(neg_revs))]
test_neg = neg_revs[int(0.75*len(neg_revs)):]
print(len(train_pos))
print(len(test_pos))
print(len(train_neg))
print(len(test_neg))

387848
479792
290886
96962
290886
96962


In [5]:
import pickle
import sys

In [9]:
stats_neg_words = {}

for doc in train_neg:
    t = set([tok for tok in word_tokenize(doc)]) ## now a word occurs only once 
    for w in t:
        if w in stats_neg_words:
            stats_neg_words[w] += 1
        else:
            stats_neg_words[w] = 1

## we divide by the number of negative docs in order to have values between 0 and 1            
for w in stats_neg_words:
    stats_neg_words[w] = stats_neg_words[w]/len(train_neg) 

## same work for positive documents
stats_pos_words = {}

for doc in train_pos:
    t = set([tok for tok in word_tokenize(doc)])
    for w in t:
        if w in stats_pos_words:
            stats_pos_words[w] += 1
        else:
            stats_pos_words[w] = 1

for w in stats_pos_words:
    stats_pos_words[w] = stats_pos_words[w]/len(train_pos) 
    

    
## we complete stats_pos_words by adding each
## word that are in stats_neg_words and not in stats_pos_words
## but with score 0
for w in stats_neg_words:
  if not w in stats_pos_words:
    stats_pos_words[w] = 0

## same work for stats_neg_words
for w in stats_pos_words:
  if not w in stats_neg_words:
    stats_neg_words[w] = 0
    
with open('positive.pkl', 'wb') as f:
    pickle.dump(stats_pos_words, f)  
with open('negative.pkl', 'wb') as f:
    pickle.dump(stats_neg_words, f)
sys.exit()    
## examples    
print(stats_pos_words["bad"])
print(stats_neg_words["bad"])
print(stats_pos_words["good"])
print(stats_neg_words["good"])

## P(w|?) ? can be positive or negative if stats is positive dictionary or negative dictionary
def score_word(w,stats):
    if w not in stats:
        return 0
    return stats[w]

## positivity of the word
def score(w,stats_p,stats_n):
    ## ATTENTION : I must take into account that, during test conditions,
    ## w can be not in the training positive docs and not int the training negative docs
    sp = score_word(w,stats_p)
    sn = score_word(w,stats_n)
    if sp+sn == 0:
        ## then, by default return 0 : not negative, not positive
        return 0
    return (sp-sn)/(sp+sn)


## positivity of a document
def eval(doc,stats_p,stats_n,threshold):
    t = set([tok for tok in word_tokenize(doc)])
    s = 0
    ## sum for all word in the document
    for w in t:
        s += score(w,stats_p,stats_n)
    ## return decision
    if s > threshold:
        return "P"
    else:
        return "N"


SystemExit: 

/home/tomine/.anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
for t in [0,1,2,3,4,5,6,7,8,9,10]:
    print("==================",t)

    tp = 0
    fp = 0
    tn = 0
    fn = 0

    for doc in test_neg:
        if eval(doc,stats_pos_words,stats_neg_words,t) == "P":
            fp += 1
        else:
            tn += 1

    for doc in test_pos:
        if eval(doc,stats_pos_words,stats_neg_words,t) == "P":
            tp += 1
        else:
            fn += 1

    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    if precision and recall:
        F1 = 2*(precision*recall)/(precision+recall)

        print("recall",recall)
        print("precision",precision)
        print("F1",F1)

================== 0
recall 0.8279222788308822
precision 0.9472883035967148
F1 0.8835921763728221
================== 1
recall 0.6613931230791444
precision 0.9844193721697752
F1 0.7912058085091946
================== 2
recall 0.4673274066128999
precision 0.9951901959061759
F1 0.63599870871756
================== 3
recall 0.31483467750252675
precision 0.9972233111198223
F1 0.47857713954253994
================== 4
recall 0.19394195664280853
precision 0.9983012156925201
F1 0.32478691525833553
================== 5
recall 0.11353932468389678
precision 0.9982771128037722
F1 0.20388924900453745
================== 6


KeyboardInterrupt: 